In [1]:
import pandas as pd
import numpy as np
import pywt
import matplotlib.pyplot as plt
from scipy import interpolate
import glob
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm


import heapq
from scipy.signal import argrelextrema

from scipy import signal
from scipy.ndimage.interpolation import shift
import pywt
from numpy.fft import fft
from numpy import zeros, floor, log10, log, mean, array, sqrt, vstack, cumsum, ones, log2, std
from numpy.linalg import svd, lstsq
import time

from sklearn.decomposition import PCA
import time
from joblib import Parallel, delayed
import multiprocessing

from sklearn.model_selection import train_test_split
import lightgbm as lgb
from catboost import CatBoostRegressor, FeaturesData, Pool
from sklearn.metrics import mean_squared_error

/Users/akuma824/Desktop/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:48: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
#==========================================================================================
#making time stamp uniform by Interpolation

def preprocess(data):
    freq=50
    ls=['X','Y','Z']
    t1=np.arange(data.Timestamp[0],data.Timestamp[(data.shape[0])-1],0.02)
    df=pd.DataFrame({'Timestamp':t1})
    for i in ls:
        fcubic = interpolate.interp1d(data.Timestamp, data[i], kind='cubic')
        df[i]=fcubic(t1)
    df.columns=['Timestamp','acc_X','acc_Y','acc_Z']
    return df

#==========================================================================================
#median filter
from scipy.signal import medfilt # import the median filter function
def median(signal):# input: numpy array 1D (one column)  
    #applying the median filter
    return  medfilt(np.array(signal), kernel_size=3) # applying the median filter order3(kernel_size=3)


#==========================================================================================
#components_selection_one_signal
import math # import math library


def components_selection_one_signal(t_signal):
    sampling_freq=50
    nyq=sampling_freq/float(2) # nyq is the nyquist frequency equal to the half of the sampling frequency[50/2= 25 Hz]

    freq1 = 0.3
    freq2 = 20

    t_signal=np.array(t_signal)
    t_signal_length=len(t_signal) # number of points in a t_signal
    
    # the t_signal in frequency domain after applying fft
    f_signal=np.fft.fft(t_signal) # 1D numpy array contains complex values (in C)
    
    # generate frequencies associated to f_signal complex values
    freqs=np.array(np.fft.fftfreq(t_signal_length, d=1/float(sampling_freq))) # frequency values between [-25hz:+25hz]
        
    df=pd.DataFrame({'freq':abs(freqs),'amplitute':f_signal})
    df['f_DC_signal']=np.where(df.freq>freq1,0,df.amplitute)
    df['f_noise_signal']=np.where(df.freq<=freq2,0,df.amplitute)
    df['f_body_signal']=np.where(df.freq<=freq1,0,np.where(df.freq>freq2,0,df.amplitute))

    
    # Inverse the transformation of signals in freq domain #
    # applying the inverse fft(ifft) to signals in freq domain and put them in float format
    t_DC_component= np.fft.ifft(np.array(df['f_DC_signal'])).real
    t_body_component= np.fft.ifft(np.array(df['f_body_signal'])).real
    t_noise=np.fft.ifft(np.array(df['f_noise_signal'])).real
    
    total_component=t_signal-t_noise # extracting the total component(filtered from noise) 
                                     #  by substracting noise from t_signal (the original signal).
    
    # return outputs mentioned earlier
    return (total_component,t_DC_component,t_body_component,t_noise) 


#=================================================================================================================
#Define verify gravity function
def mag_3_signals(df): # Euclidian magnitude
    return np.array(np.sqrt(np.square(df).sum(axis=1)))

def verify_gravity(data):
    
    acc_x=np.array(data['acc_X']) # copy acc_X column from dataframe in raw_dic having the key mentioned above
    acc_y=np.array(data['acc_Y'])# copy acc_Y column  from dataframe in raw_dic having the key mentioned above
    acc_z=np.array(data['acc_Z'])# copy acc_Z column  from dataframe in raw_dic having the key mentioned above

    # apply the filtering method to acc_[X,Y,Z] and store gravity components
    grav_acc_X=components_selection_one_signal(acc_x)[1] 
    grav_acc_Y=components_selection_one_signal(acc_y)[1]
    grav_acc_Z=components_selection_one_signal(acc_z)[1]
    
    # calculating gravity magnitude signal
    grav_acc_mag=mag_3_signals(grav_acc_X, grav_acc_Y,grav_acc_Z)
    print('mean value = ',round((sum(grav_acc_mag) / len(grav_acc_mag)),3),' g')
    
#=================================================================================================================    
#Define jerking and magnitude functions
def jerk_one_signal(signal):
    signal=pd.DataFrame(signal)
    jerk=(signal.shift(-1)-signal)/0.02
    return np.array(jerk.dropna()).transpose()[0]

def time_domain_signal(data):
    time_sig_df=pd.DataFrame()
    for column in ['acc_X','acc_Y','acc_Z']:
        t_signal=np.array(data[column])
        #med_filtred=median(t_signal)
        med_filtred=(t_signal)
        _,grav_acc,body_acc,_=components_selection_one_signal(med_filtred)
        body_acc_jerk=jerk_one_signal(body_acc)
        time_sig_df['t_body_'+column]=body_acc[:-1]
        time_sig_df['t_grav_'+column]= grav_acc[:-1]
        time_sig_df['t_body_acc_jerk_'+column[-1]]=body_acc_jerk

    # all 15 axial signals generated above are reordered to facilitate magnitudes signals generation
    new_columns_ordered=['t_body_acc_X','t_body_acc_Y','t_body_acc_Z',
                              't_grav_acc_X','t_grav_acc_Y','t_grav_acc_Z',
                              't_body_acc_jerk_X','t_body_acc_jerk_Y','t_body_acc_jerk_Z']


    # create new dataframe to order columns
    time_sig_df=time_sig_df[new_columns_ordered]

    # Magnitude Features
    for i in range(0,9,3):
        mag_col_name=new_columns_ordered[i][:-1]+'mag'# Create the magnitude column name related to each 3-axial signals
        time_sig_df[mag_col_name]=mag_3_signals(time_sig_df[new_columns_ordered[i:i+3]]) # store the signal_mag with its appropriate column name

    return(time_sig_df)

In [3]:
a=glob.glob("training_data/*.csv")

In [4]:
data=preprocess(pd.read_csv(a[14]))
print(data.shape)
data.head()

(60000, 4)


,Timestamp,acc_X,acc_Y,acc_Z
0,0.00,0.186035,0.170654,-0.878174
1,0.02,0.148438,0.209961,-0.894287
2,0.04,0.166016,0.219727,-0.925049
3,0.06,0.206299,0.194092,-0.926758
4,0.08,0.173584,0.204102,-0.983154


In [5]:
time_sig_df=time_domain_signal(data)
time_sig_df.shape

(59999, 12)

In [6]:
time_sig_df.head()

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_acc_mag,t_grav_acc_mag,t_body_acc_jerk_mag
0,0.051701,-0.309492,-0.430242,0.131715,0.558750,-0.319212,-1.716811,-4.916696,-12.082357,0.532510,0.656846,13.156925
1,0.017365,-0.407826,-0.671889,0.131556,0.549420,-0.335519,0.975826,6.859124,9.164233,0.786167,0.657071,11.488384
2,0.036881,-0.270644,-0.488604,0.131486,0.540100,-0.351798,1.638469,-4.648829,-5.982242,0.559770,0.657844,7.751349
3,0.069651,-0.363620,-0.608249,0.131505,0.530795,-0.368040,-1.029729,2.558821,1.223206,0.712066,0.659159,3.017307
4,0.049056,-0.312444,-0.583785,0.131613,0.521510,-0.384237,2.517675,1.077161,-0.436842,0.663952,0.661009,2.773048


In [7]:
#Fractal Dimension using Katz FD algorithm
def katz(data):
    n = len(data)-1
    L = np.hypot(np.diff(data), 1).sum() # Sum of distances
    d = np.hypot(data - data[0], np.arange(len(data))).max() # furthest distance from first point
    return np.log10(n) / (np.log10(d/L) + np.log10(n))




#Coefficient of variation
def coeff_var(a):
    output = np.std(a)/np.mean(a) #computing coefficient of variation
    return output

#Mean and variance of Vertex to Vertex Slope
def slope(x):
    
    amp_max = np.array(x[argrelextrema(np.array(x), np.greater)[0]])
    t_max = argrelextrema(np.array(x), np.greater)[0]
    amp_min = np.array(x[argrelextrema(np.array(x), np.less)[0]])
    t_min = argrelextrema(np.array(x), np.less)[0]
    t = np.concatenate((t_max,t_min),axis=0)
    t.sort()#sort on the basis of time

    amp = np.zeros(len(t))
    res = np.zeros(len(t))
    
    for l in range(len(t)):
        amp[l]=x[t[l]]

    amp_diff = first_diff(amp)
    t_diff = first_diff(t)

    for q in range(len(amp_diff)):
        res[q] = amp_diff[q]/t_diff[q]         
    
    res=res[~np.isnan(res)]
    return [np.mean(res),np.std(res)] #returning mean and std of vertex to vertex slope


#Hjorth Parameter
def hjorth(input):                                              
    hfeatures = []
    diff_input = np.diff(input)
    diff_diffinput = np.diff(diff_input)
    
    hjorth_activity = np.var(input)
    hjorth_mobility = np.sqrt(np.var(diff_input)/hjorth_activity)
    hjorth_diffmobility = np.sqrt(np.var(diff_diffinput)/np.var(diff_input))
    hjorth_complexity = hjorth_diffmobility/hjorth_mobility
    
    hfeatures.append(hjorth_activity)
    hfeatures.append(hjorth_mobility)
    hfeatures.append(hjorth_complexity)
    
    return hfeatures  #returning hjorth activity, hjorth mobility , hjorth complexity


#Kurtosis
def kurtosis(a):
    mean_i = np.mean(a) # Saving the mean of array i
    std_i = np.std(a) # Saving the standard deviation of array i
    t = 0.0
    for j in a:
        t += (pow((j-mean_i)/std_i,4)-3)
    kurtosis_i = t/len(a) # Formula: (1/N)*(summation(x_i-mean)/standard_deviation)^4-3
    return kurtosis_i


#Second difference Mean,Max,std
def sec_diff(b):
    temp1 = abs(b-b.shift(1)) # Obtaining the 1st Diffs
    t = abs(temp1-temp1.shift(1)) # Summing the 2nd Diffs
    output = t.mean() # Calculating the mean of the 2nd Diffs
    return [t.mean(),t.max(),t.std()]

#Skewness
def skewness(a):
    import scipy.stats as sp
    skew_array=sp.stats.skew(a,axis=0,bias=True)
    return skew_array #returning skewness


#First Difference Mean,Max,std
def first_diff_mean(a):
    output = abs(a-a.shift(1)) # Obtaining the 1st Diffs
    return [output.mean(),output.max(),output.std()] #returns first diff mean,max,min

#First Difference
def first_diff(a):
    if str(type(a))!="<class 'pandas.core.series.Series'>":
        a=pd.DataFrame(a)[0]
    output = a-a.shift(1)# Obtaining the 1st Diffs
    return output


#wavelet features
def wavelet_features(epoch): 
    wfeatures = []
    
    #calculating the coefficients of wavelet transform.
    cA_values,cD_values=pywt.dwt(epoch,'coif1')
    
    cA_square=np.array([x for x in (np.square(cA_values)).tolist() if x != 0])
    cD_square=np.array([x for x in (np.square(cD_values)).tolist() if x != 0])
    
      
    wfeatures.append(np.mean(cA_values)) #cA_mean
    wfeatures.append(abs(np.std(cA_values))) #cA_std
    wfeatures.append(abs(np.sum(np.square(cA_values)))) #cA_Energy
    wfeatures.append(abs(np.sum(cA_square * np.log(cA_square)))) #Entropy_A
    
    wfeatures.append(np.mean(cD_values)) #cD_mean
    wfeatures.append(abs(np.std(cD_values))) #cD_std
    wfeatures.append(abs(np.sum(np.square(cD_values)))) #cD_Energy
    wfeatures.append(abs(np.sum(cD_square * np.log(cD_square)))) #Entropy_D

    return wfeatures # returning 'Wavelet Approximate Mean','Wavelet Approximate Std Deviation','Wavelet Approximate Energy','Wavelet Approximate Entropy','Wavelet Detailed Mean','Wavelet Detailed Std Deviation','Wavelet Detailed Energy','Wavelet Detailed Entropy'


In [8]:
time_sig_df.head()

,t_body_acc_X,t_body_acc_Y,t_body_acc_Z,t_grav_acc_X,t_grav_acc_Y,t_grav_acc_Z,t_body_acc_jerk_X,t_body_acc_jerk_Y,t_body_acc_jerk_Z,t_body_acc_mag,t_grav_acc_mag,t_body_acc_jerk_mag
0,0.051701,-0.309492,-0.430242,0.131715,0.558750,-0.319212,-1.716811,-4.916696,-12.082357,0.532510,0.656846,13.156925
1,0.017365,-0.407826,-0.671889,0.131556,0.549420,-0.335519,0.975826,6.859124,9.164233,0.786167,0.657071,11.488384
2,0.036881,-0.270644,-0.488604,0.131486,0.540100,-0.351798,1.638469,-4.648829,-5.982242,0.559770,0.657844,7.751349
3,0.069651,-0.363620,-0.608249,0.131505,0.530795,-0.368040,-1.029729,2.558821,1.223206,0.712066,0.659159,3.017307
4,0.049056,-0.312444,-0.583785,0.131613,0.521510,-0.384237,2.517675,1.077161,-0.436842,0.663952,0.661009,2.773048


In [9]:
def features_generation(t_window):
    
    # select mag columns : the last 3 columns in a time domain window
    
    mag_columns=t_window.columns # mag columns' names
    
    t_mag_features=[] # a global list will contain all time domain magnitude features
    
    for col in mag_columns: # iterate throw each mag column
        
        fkatz = [katz(t_window[col])] # 1 value
        fcoeff_var   = [coeff_var(t_window[col])] # 1 value
        fslope    = slope(t_window[col])# 2 value
        fhjorth    = hjorth(t_window[col])# 3 value
        fsec_diff    = sec_diff(t_window[col])# 3 value
        ffirst_diff_mean = first_diff_mean(t_window[col])# 3 value
        fwavelet_features    = wavelet_features(t_window[col])# 8 value
        
        # 13 value per each t_mag_column
        col_mag_values = fkatz+fcoeff_var+fslope+fhjorth+fsec_diff+ffirst_diff_mean+fwavelet_features
        
        # col_mag_values will be added to the global list
        t_mag_features= t_mag_features+ col_mag_values
    
    # t_mag_features contains 65 values = 13 values (per each t_mag_column) x 5 (t_mag_columns)
    return t_mag_features
 
def Dataset_Generation_PipeLine(b):
    data=preprocess(pd.read_csv(b))
    time_sig_df=time_domain_signal(data)
    # concatenate all features and append the activity id and the user id
    row= features_generation(time_sig_df)
    return(row)


In [10]:
def features_names():
    # Generating time feature names
    
    # time domain magnitude signals' names
    magnitude_signals=['t_body_acc_X','t_body_acc_Y','t_body_acc_Z',
                       't_grav_acc_X','t_grav_acc_Y','t_grav_acc_Z',
                       't_body_acc_jerk_X','t_body_acc_jerk_Y','t_body_acc_jerk_Z',
                       't_body_acc_Mag','t_grav_acc_Mag','t_body_acc_jerk_Mag']

    # functions' names:
    t_one_input_features_name1=['_katz()','_coeff_var()']

    t_one_input_features_slope=['_slope_mean()','_slope_std()']

    t_one_input_features_hjorth=['_hjorth_activity()','_hjorth_mobility()','_hjorth_complexity()']

    t_one_input_features_sec_diff=['_sec_diff_mean()','_sec_diff_max()','_sec_diff_std()']
    t_one_input_features_first_diff=['_first_diff_mean()','_first_diff_max()','_first_diff_std()']
    
    t_one_input_features_wavelet=['_wavelet_cA_mean()','_wavelet_cA_std()','_wavelet_cA_Energy()','_wavelet_Entropy_A()',
                                  '_wavelet_cD_mean()','_wavelet_cD_std()','_wavelet_cD_Energy()','_wavelet_Entropy_D()']
    
    features=[]# Empty list : it will contain all time domain features' names
    
    for columns in magnitude_signals: # iterate throw time domain magnitude column names

        # build feature names related to that column
        #list 1
        for feature in t_one_input_features_name1:
            newcolumn=columns+feature
            features.append(newcolumn)
        
        # list 2
        for feature in t_one_input_features_slope: 
            newcolumn=columns+feature
            features.append(newcolumn)
            
       
        # list 3
        for feature in t_one_input_features_hjorth:
            newcolumn=columns+feature
            features.append(newcolumn)
            
        # list 4
        for feature in t_one_input_features_sec_diff:
            newcolumn=columns+feature
            features.append(newcolumn)
        # list 5
        for feature in t_one_input_features_first_diff:
            newcolumn=columns+feature
            features.append(newcolumn)
        # list 6
        for feature in t_one_input_features_wavelet:
            newcolumn=columns+feature
            features.append(newcolumn)
            
        
    ###########################################################################################################
    time_list_features=features
    
    return time_list_features # return all time domain features' names

In [11]:
len(features_names())

252

In [13]:
from joblib import Parallel, delayed
import multiprocessing
import time

### training

In [14]:
a=glob.glob("training_data/*.csv")
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine)(i) for i in a)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#47.693 min 

--- 47.6933491508166 Mins ---


In [15]:
training_data=pd.DataFrame(result)
training_data.columns=features_names()
training_data['measurement_id']=[item[len('training_data/'):-4] for item in a]
print(training_data.shape) #(1858, 253)
training_data.head()

(1858, 253)


,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),t_body_acc_X_sec_diff_std(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,1.000085,-29974.852208,0.000111,0.040033,0.009989,0.434612,2.920552,0.018337,0.657910,0.030628,...,3.184254,5.346787,6.338059,2.062911e+06,1.145451e+07,0.036098,2.634289,208237.389285,801308.169757,4d9e9934-8c16-439b-934c-c0f46c0e043a
1,1.000028,-15309.058182,0.000019,0.023218,0.004429,0.376565,3.444969,0.007284,0.770953,0.018309,...,1.952803,2.175123,3.867517,5.907048e+05,3.539394e+06,0.004280,1.450144,63092.265393,309719.065699,26f5e58c-ab76-42e1-a252-43ed459a4ad7
2,1.000041,-24476.126809,0.000159,0.031002,0.001438,0.808562,1.745299,0.012315,1.035255,0.022183,...,1.371653,2.302094,2.918124,4.144801e+05,1.949575e+06,0.004687,1.102467,36466.075979,105559.633093,33928414-f322-43ca-b2b1-9152c2c72906
3,1.000063,20859.377684,-0.000101,0.030298,0.015893,0.296749,3.490893,0.014368,0.555784,0.023123,...,1.952162,3.698664,4.338665,9.751889e+05,5.049100e+06,0.009293,1.602418,77040.036393,256708.969107,2a72fb3e-89a1-41cc-a2bd-c8d110fd4b2d
4,1.000013,-68508.316731,-0.000046,0.013504,0.001439,0.442898,2.610914,0.004767,0.397376,0.012030,...,0.755281,0.987380,1.732783,1.193277e+05,4.447511e+05,-0.000516,0.569863,9742.638730,16869.528313,dcb79a9e-e481-4d8c-9ea6-d5bd450386e7


In [16]:
#ancillary
b=glob.glob("ancillary_data/*.csv")
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine)(i) for i in b)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#12.124 Mins

--- 12.124863918622335 Mins ---


In [17]:
ancillary_data=pd.DataFrame(result)
ancillary_data.columns=features_names()
ancillary_data['measurement_id']=[item[len('ancillary_data/'):-4] for item in b]
print(ancillary_data.shape)
ancillary_data.head()

(352, 253)


,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),t_body_acc_X_sec_diff_std(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,1.000102,-23794.785574,0.000152,0.049220,0.009667,0.489939,2.708070,0.019744,1.266837,0.033143,...,2.548128,4.977219,4.990713,1.490497e+06,7.962460e+06,0.025361,2.157240,139639.099586,511297.241987,05b3f5ec-24aa-4558-b19f-7ab82cd3f1d3
1,1.000020,248126.607933,0.000029,0.021591,0.002747,0.408588,3.231427,0.007975,0.621162,0.014041,...,1.482954,2.399206,2.676516,3.876237e+05,1.661241e+06,0.000614,1.161024,40441.978670,129566.697988,248e1db4-0919-4721-b11a-2b979a60ff11
2,1.000048,92744.821964,-0.000097,0.032564,0.005323,0.467950,2.794638,0.007522,2.216589,0.028506,...,1.899752,1.728611,4.010005,5.720852e+05,3.552519e+06,0.007961,1.327720,52890.674707,241978.425127,eb95eefd-b52c-45dd-a206-7122eeac085f
3,1.000041,-22129.092108,0.000031,0.031849,0.004251,0.470700,3.041384,0.009925,1.301723,0.021622,...,1.967826,2.706481,3.960815,6.904158e+05,3.583692e+06,0.009733,1.530862,70311.297770,251959.592370,fae9e929-899a-4fe6-a3de-4fc96364611c
4,1.000022,-184724.196053,0.000014,0.026206,0.001299,0.627182,2.470087,0.005834,0.910777,0.017934,...,1.851179,1.321943,2.857269,2.973554e+05,1.884685e+06,0.010857,1.253204,47120.690834,294557.885860,dcca711d-e105-400a-8389-de6c10bbd209


In [18]:
data=training_data.append(pd.DataFrame(ancillary_data),ignore_index=True)
print(data.shape)
data.head()

(2210, 253)


,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),t_body_acc_X_sec_diff_std(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,1.000085,-29974.852208,0.000111,0.040033,0.009989,0.434612,2.920552,0.018337,0.657910,0.030628,...,3.184254,5.346787,6.338059,2.062911e+06,1.145451e+07,0.036098,2.634289,208237.389285,801308.169757,4d9e9934-8c16-439b-934c-c0f46c0e043a
1,1.000028,-15309.058182,0.000019,0.023218,0.004429,0.376565,3.444969,0.007284,0.770953,0.018309,...,1.952803,2.175123,3.867517,5.907048e+05,3.539394e+06,0.004280,1.450144,63092.265393,309719.065699,26f5e58c-ab76-42e1-a252-43ed459a4ad7
2,1.000041,-24476.126809,0.000159,0.031002,0.001438,0.808562,1.745299,0.012315,1.035255,0.022183,...,1.371653,2.302094,2.918124,4.144801e+05,1.949575e+06,0.004687,1.102467,36466.075979,105559.633093,33928414-f322-43ca-b2b1-9152c2c72906
3,1.000063,20859.377684,-0.000101,0.030298,0.015893,0.296749,3.490893,0.014368,0.555784,0.023123,...,1.952162,3.698664,4.338665,9.751889e+05,5.049100e+06,0.009293,1.602418,77040.036393,256708.969107,2a72fb3e-89a1-41cc-a2bd-c8d110fd4b2d
4,1.000013,-68508.316731,-0.000046,0.013504,0.001439,0.442898,2.610914,0.004767,0.397376,0.012030,...,0.755281,0.987380,1.732783,1.193277e+05,4.447511e+05,-0.000516,0.569863,9742.638730,16869.528313,dcb79a9e-e481-4d8c-9ea6-d5bd450386e7


In [25]:
data.to_csv('cispd_comp_training_abhiroop_lastfeatures.csv',index=False)

### test

In [19]:
a=glob.glob("testing_data/*.csv")
start_time = time.time()
num_cores = multiprocessing.cpu_count()
result=Parallel(n_jobs=num_cores)(delayed(Dataset_Generation_PipeLine)(i) for i in a)
print("--- %s Mins ---" % ((time.time() - start_time)/60))
#18.773 min 

testing_data=pd.DataFrame(result)
testing_data.columns=features_names()
testing_data['measurement_id']=[item[len('testing_data/'):-4] for item in a]
print(testing_data.shape) #(1858, 253)
testing_data.head()

--- 18.773693362871807 Mins ---
(618, 253)


,t_body_acc_X_katz(),t_body_acc_X_coeff_var(),t_body_acc_X_slope_mean(),t_body_acc_X_slope_std(),t_body_acc_X_hjorth_activity(),t_body_acc_X_hjorth_mobility(),t_body_acc_X_hjorth_complexity(),t_body_acc_X_sec_diff_mean(),t_body_acc_X_sec_diff_max(),t_body_acc_X_sec_diff_std(),...,t_body_acc_jerk_Mag_first_diff_std(),t_body_acc_jerk_Mag_wavelet_cA_mean(),t_body_acc_jerk_Mag_wavelet_cA_std(),t_body_acc_jerk_Mag_wavelet_cA_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_A(),t_body_acc_jerk_Mag_wavelet_cD_mean(),t_body_acc_jerk_Mag_wavelet_cD_std(),t_body_acc_jerk_Mag_wavelet_cD_Energy(),t_body_acc_jerk_Mag_wavelet_Entropy_D(),measurement_id
0,1.000063,79425.921640,-0.000002,0.033262,0.009041,0.398230,3.271127,0.011460,0.884043,0.027248,...,2.522197,2.833457,5.230665,1.061686e+06,6.527228e+06,0.002004,1.840900,101670.941446,4.664711e+05,4bcb3a76-9009-4afd-ad05-1f472e142dbe
1,1.000091,-18491.243253,0.000113,0.037640,0.012078,0.412876,2.642028,0.014033,1.184483,0.030546,...,1.808095,2.908805,4.221867,7.885856e+05,4.214891e+06,0.009767,1.401592,58938.620242,1.935921e+05,c5958651-8695-4d6b-b8c5-9eb6c6866078
2,1.000383,-37022.676846,-0.000482,0.092571,0.038706,0.492713,2.614953,0.033951,1.948639,0.068043,...,5.750095,8.295351,11.743038,6.201767e+06,4.662059e+07,-0.016214,4.445950,593041.569770,3.578705e+06,7598e18a-79c2-4936-a951-b9aedae10d31
3,1.000128,-7829.415014,-0.000109,0.056130,0.004401,0.820201,1.915778,0.024805,1.901673,0.033478,...,2.892777,6.503091,5.557133,1.137311e+06,6.427446e+06,0.021674,2.624112,107035.807864,3.910691e+05,b31841bf-dce6-4cb1-9389-2b3d2810a9ea
4,1.000005,5268.378514,-0.000006,0.008599,0.000920,0.345503,3.508316,0.002739,0.341342,0.008055,...,0.630309,0.694806,1.468830,7.921182e+04,3.201905e+05,0.000460,0.472556,6699.726661,1.235729e+04,3f2382de-6aba-49e1-b4c2-fc4f0498e9ca


In [26]:
testing_data.to_csv('cispd_comp_testing_abhiroop_lastfeatures.csv',index=False)

## Demographic Data

In [344]:
cis_pd_demo=pd.read_csv('clinical_data/CIS-PD_Demographics.csv')
cis_pd_demo=cis_pd_demo.drop(['Race','Ethnicity'],axis=1)
cis_pd_demo['Gender']=cis_pd_demo['Gender'].map({'Male':1,'Female':0})
print(cis_pd_demo.shape)
cis_pd_demo.head()

(21, 3)


,subject_id,Age,Gender
0,1000,63,1
1,1004,52,1
2,1006,71,1
3,1007,75,1
4,1016,66,0


In [346]:
cis_pd_updrs_part1_2_4=pd.read_csv('clinical_data/CIS-PD_UPDRS_Part1_2_4.csv')
cis_pd_updrs_part1_2_4.drop('Visit',axis=1,inplace=True)
print(cis_pd_updrs_part1_2_4.shape)
cis_pd_updrs_part1_2_4.head()

(21, 9)


,subject_id,UPDRS_PartI_Total,UPDRS_PartII_Total,UPDRS_4.1,UPDRS_4.2,UPDRS_4.3,UPDRS_4.4,UPDRS_4.5,UPDRS_4.6
0,1000,4,5,0,0,1,3,1,1
1,1004,13,6,1,3,1,4,3,0
2,1006,10,19,0,0,1,0,2,0
3,1007,18,9,0,0,2,3,1,2
4,1016,15,2,0,0,0,0,0,0


In [350]:
cis_pd_updrs_part3=pd.read_csv('clinical_data/CIS-PD_UPDRS_Part3.csv')
cis_pd_updrs_part3.head()

,subject_id,Visit,ParticipantState,UPDRS_3a,UPDRS_3b,UPDRS_3c,UPDRS_3C1,UPDRS_3.1,UPDRS_3.2,UPDRS_3.3 Neck,UPDRS_3.3 Right Upper Extremity,UPDRS_3.3 Left Upper Extremity,UPDRS_3.3 Right Lower Extremity,UPDRS_3.3 Left Lower Extremity,UPDRS_3.4 Right Hand,UPDRS_3.4 Left Hand,UPDRS_3.5 Right Hand,UPDRS_3.5 Left Hand,UPDRS_3.6 Right Hand,UPDRS_3.6 Left Hand,UPDRS_3.7 Right Foot,UPDRS_3.7 Left Foot,UPDRS_3.8 Right Leg,UPDRS_3.8 Left Leg,UPDRS_3.9,UPDRS_3.10,UPDRS_3.11,UPDRS_3.12,UPDRS_3.13,UPDRS_3.14,UPDRS_3.15 Right Hand,UPDRS_3.15 Left Hand,UPDRS_3.16 Right Hand,UPDRS_3.16 Left Hand,UPDRS_3.17 Right Upper Extremity,UPDRS_3.17 Left Upper Extremity,UPDRS_3.17 Right Lower Extremity,UPDRS_3.17 Left Lower Extremity,UPDRS_3.17 Lip-Jaw,UPDRS_3.18,UPDRS_3.19A,UPDRS_3.19B,UPDRS_3.20
0,1000,Baseline,NaN,Yes,On,Yes,115.0,1.0,2.0,2.0,2.0,3.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,Yes,No,2.0
1,1004,2 Weeks: Time 0,Off,NaN,NaN,NaN,NaN,1.0,1.0,0.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,No,NaN,2.0
2,1004,2 Weeks: Time 60,On,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Yes,No,2.0
3,1006,Baseline,NaN,Yes,On,Yes,150.0,1.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,No,NaN,2.0
4,1007,2 Weeks: Time 0,Off,NaN,NaN,NaN,NaN,0.0,0.0,2.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,1.0,2.0,1.0,3.0,3.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,2.0,No,NaN,2.0


In [347]:
cis_pd_updrs_part3=pd.read_csv('clinical_data/CIS-PD_UPDRS_Part3.csv')
cis_pd_updrs_part3.drop(['UPDRS_3a','UPDRS_3b','UPDRS_3c','UPDRS_3C1','ParticipantState','UPDRS_3.19B'],axis=1,inplace=True)

visit1=cis_pd_updrs_part3.Visit.map({'Baseline':0,'2 Weeks: Time 0':1,'2 Weeks: Time 60':1})
visit2=cis_pd_updrs_part3.Visit.map({'Baseline':0,'2 Weeks: Time 0':0,'2 Weeks: Time 60':1})

col=[e for e in cis_pd_updrs_part3.columns if e not in ('subject_id','Visit')]
cis_pd_updrs_part3=cis_pd_updrs_part3.groupby(['subject_id'])[col].ffill().bfill()

cis_pd_updrs_part3['Visit']=visit2
cis_pd_updrs_part3['Visit_Type']=visit1
cis_pd_updrs_part3['UPDRS_3.19A']=cis_pd_updrs_part3['UPDRS_3.19A'].map({'Yes':1,'No':0})

cis_pd_updrs_part3['Rigidity']=cis_pd_updrs_part3[['UPDRS_3.3 Neck','UPDRS_3.3 Right Upper Extremity','UPDRS_3.3 Left Upper Extremity','UPDRS_3.3 Right Lower Extremity','UPDRS_3.3 Left Lower Extremity']].mean(axis=1)
cis_pd_updrs_part3['Tapping']=cis_pd_updrs_part3[['UPDRS_3.4 Right Hand','UPDRS_3.4 Left Hand','UPDRS_3.7 Right Foot','UPDRS_3.7 Left Foot']].mean(axis=1)
cis_pd_updrs_part3['Body Tremor']=cis_pd_updrs_part3[['UPDRS_3.15 Right Hand','UPDRS_3.15 Left Hand','UPDRS_3.16 Right Hand','UPDRS_3.16 Left Hand','UPDRS_3.17 Right Upper Extremity','UPDRS_3.17 Left Upper Extremity','UPDRS_3.17 Right Lower Extremity','UPDRS_3.17 Left Lower Extremity','UPDRS_3.17 Lip-Jaw']].mean(axis=1)
cis_pd_updrs_part3['Movement']=cis_pd_updrs_part3[['UPDRS_3.5 Right Hand','UPDRS_3.5 Left Hand','UPDRS_3.6 Right Hand','UPDRS_3.6 Left Hand']].mean(axis=1)

type1=cis_pd_updrs_part3.copy()
type1.drop(['Visit','Visit_Type'],axis=1,inplace=True)
numeric_cols = list(set(type1.columns) - set(['subject_id']))
type1[numeric_cols] += 1

type2=type1.groupby('subject_id').pct_change()
type2['subject_id']=cis_pd_updrs_part3['subject_id']
type2=type2.groupby('subject_id').max().reset_index().fillna(0)
type2.columns=['subject_id']+list('PerChange_'+type2.columns[1:])
cis_pd_updrs_part3=pd.merge(cis_pd_updrs_part3[cis_pd_updrs_part3.Visit==0].drop(['Visit'],axis=1),type2,on='subject_id')
print(cis_pd_updrs_part3.shape)
cis_pd_updrs_part3.head()


(21, 80)


,subject_id,UPDRS_3.1,UPDRS_3.2,UPDRS_3.3 Neck,UPDRS_3.3 Right Upper Extremity,UPDRS_3.3 Left Upper Extremity,UPDRS_3.3 Right Lower Extremity,UPDRS_3.3 Left Lower Extremity,UPDRS_3.4 Right Hand,UPDRS_3.4 Left Hand,UPDRS_3.5 Right Hand,UPDRS_3.5 Left Hand,UPDRS_3.6 Right Hand,UPDRS_3.6 Left Hand,UPDRS_3.7 Right Foot,UPDRS_3.7 Left Foot,UPDRS_3.8 Right Leg,UPDRS_3.8 Left Leg,UPDRS_3.9,UPDRS_3.10,UPDRS_3.11,UPDRS_3.12,UPDRS_3.13,UPDRS_3.14,UPDRS_3.15 Right Hand,UPDRS_3.15 Left Hand,UPDRS_3.16 Right Hand,UPDRS_3.16 Left Hand,UPDRS_3.17 Right Upper Extremity,UPDRS_3.17 Left Upper Extremity,UPDRS_3.17 Right Lower Extremity,UPDRS_3.17 Left Lower Extremity,UPDRS_3.17 Lip-Jaw,UPDRS_3.18,UPDRS_3.19A,UPDRS_3.20,Visit_Type,Rigidity,Tapping,Body Tremor,Movement,PerChange_UPDRS_3.1,PerChange_UPDRS_3.2,PerChange_UPDRS_3.3 Neck,PerChange_UPDRS_3.3 Right Upper Extremity,PerChange_UPDRS_3.3 Left Upper Extremity,PerChange_UPDRS_3.3 Right Lower Extremity,PerChange_UPDRS_3.3 Left Lower Extremity,PerChange_UPDRS_3.4 Right Hand,PerChange_UPDRS_3.4 Left Hand,PerChange_UPDRS_3.5 Right Hand,PerChange_UPDRS_3.5 Left Hand,PerChange_UPDRS_3.6 Right Hand,PerChange_UPDRS_3.6 Left Hand,PerChange_UPDRS_3.7 Right Foot,PerChange_UPDRS_3.7 Left Foot,PerChange_UPDRS_3.8 Right Leg,PerChange_UPDRS_3.8 Left Leg,PerChange_UPDRS_3.9,PerChange_UPDRS_3.10,PerChange_UPDRS_3.11,PerChange_UPDRS_3.12,PerChange_UPDRS_3.13,PerChange_UPDRS_3.14,PerChange_UPDRS_3.15 Right Hand,PerChange_UPDRS_3.15 Left Hand,PerChange_UPDRS_3.16 Right Hand,PerChange_UPDRS_3.16 Left Hand,PerChange_UPDRS_3.17 Right Upper Extremity,PerChange_UPDRS_3.17 Left Upper Extremity,PerChange_UPDRS_3.17 Right Lower Extremity,PerChange_UPDRS_3.17 Left Lower Extremity,PerChange_UPDRS_3.17 Lip-Jaw,PerChange_UPDRS_3.18,PerChange_UPDRS_3.19A,PerChange_UPDRS_3.20,PerChange_Rigidity,PerChange_Tapping,PerChange_Body Tremor,PerChange_Movement
0,1000,1.0,2.0,2.0,2.0,3.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.0,0,2.4,1.25,0.444444,1.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000
1,1004,1.0,1.0,0.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,1,1.0,2.00,0.444444,1.50,0.0,0.0,0.000000,0.0,-0.333333,0.0,-0.5,0.000000,-0.333333,0.000000,-0.333333,-0.5,-0.666667,-0.333333,-0.25,-0.5,-0.333333,0.0,-0.5,0.0,0.0,0.0,-0.333333,-0.5,-0.5,-0.5,-0.5,0.0,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0,-0.2,-0.250000,-0.307692,-0.400
2,1006,1.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0,2.0,0,0.4,1.25,0.222222,1.25,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000
3,1007,0.0,0.0,2.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,1.0,2.0,1.0,3.0,3.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,2.0,0,2.0,1,1.0,2.50,0.555556,1.50,1.0,0.0,-0.666667,-0.5,-0.500000,-0.5,0.0,-0.333333,0.000000,-0.333333,0.000000,0.0,-0.500000,-0.750000,-0.25,0.0,0.500000,0.0,0.0,0.0,0.0,-0.5,-0.333333,0.0,0.0,-0.5,-0.5,-0.5,0.0,-0.333333,0.0,0.0,-0.333333,0.0,0.0,-0.5,-0.357143,-0.285714,-0.200
4,1016,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,1,0.0,1.00,0.111111,1.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,-0.500000,-0.500000,0.5,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,2.0,0.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.100000,-0.125


In [348]:
clinical_data=pd.merge(cis_pd_demo,cis_pd_updrs_part1_2_4,on='subject_id')
clinical_data=pd.merge(clinical_data,cis_pd_updrs_part3,on='subject_id')
print(clinical_data.shape)
clinical_data.head()

(21, 90)


,subject_id,Age,Gender,UPDRS_PartI_Total,UPDRS_PartII_Total,UPDRS_4.1,UPDRS_4.2,UPDRS_4.3,UPDRS_4.4,UPDRS_4.5,UPDRS_4.6,UPDRS_3.1,UPDRS_3.2,UPDRS_3.3 Neck,UPDRS_3.3 Right Upper Extremity,UPDRS_3.3 Left Upper Extremity,UPDRS_3.3 Right Lower Extremity,UPDRS_3.3 Left Lower Extremity,UPDRS_3.4 Right Hand,UPDRS_3.4 Left Hand,UPDRS_3.5 Right Hand,UPDRS_3.5 Left Hand,UPDRS_3.6 Right Hand,UPDRS_3.6 Left Hand,UPDRS_3.7 Right Foot,UPDRS_3.7 Left Foot,UPDRS_3.8 Right Leg,UPDRS_3.8 Left Leg,UPDRS_3.9,UPDRS_3.10,UPDRS_3.11,UPDRS_3.12,UPDRS_3.13,UPDRS_3.14,UPDRS_3.15 Right Hand,UPDRS_3.15 Left Hand,UPDRS_3.16 Right Hand,UPDRS_3.16 Left Hand,UPDRS_3.17 Right Upper Extremity,UPDRS_3.17 Left Upper Extremity,UPDRS_3.17 Right Lower Extremity,UPDRS_3.17 Left Lower Extremity,UPDRS_3.17 Lip-Jaw,UPDRS_3.18,UPDRS_3.19A,UPDRS_3.20,Visit_Type,Rigidity,Tapping,Body Tremor,Movement,PerChange_UPDRS_3.1,PerChange_UPDRS_3.2,PerChange_UPDRS_3.3 Neck,PerChange_UPDRS_3.3 Right Upper Extremity,PerChange_UPDRS_3.3 Left Upper Extremity,PerChange_UPDRS_3.3 Right Lower Extremity,PerChange_UPDRS_3.3 Left Lower Extremity,PerChange_UPDRS_3.4 Right Hand,PerChange_UPDRS_3.4 Left Hand,PerChange_UPDRS_3.5 Right Hand,PerChange_UPDRS_3.5 Left Hand,PerChange_UPDRS_3.6 Right Hand,PerChange_UPDRS_3.6 Left Hand,PerChange_UPDRS_3.7 Right Foot,PerChange_UPDRS_3.7 Left Foot,PerChange_UPDRS_3.8 Right Leg,PerChange_UPDRS_3.8 Left Leg,PerChange_UPDRS_3.9,PerChange_UPDRS_3.10,PerChange_UPDRS_3.11,PerChange_UPDRS_3.12,PerChange_UPDRS_3.13,PerChange_UPDRS_3.14,PerChange_UPDRS_3.15 Right Hand,PerChange_UPDRS_3.15 Left Hand,PerChange_UPDRS_3.16 Right Hand,PerChange_UPDRS_3.16 Left Hand,PerChange_UPDRS_3.17 Right Upper Extremity,PerChange_UPDRS_3.17 Left Upper Extremity,PerChange_UPDRS_3.17 Right Lower Extremity,PerChange_UPDRS_3.17 Left Lower Extremity,PerChange_UPDRS_3.17 Lip-Jaw,PerChange_UPDRS_3.18,PerChange_UPDRS_3.19A,PerChange_UPDRS_3.20,PerChange_Rigidity,PerChange_Tapping,PerChange_Body Tremor,PerChange_Movement
0,1000,63,1,4,5,0,0,1,3,1,1,1.0,2.0,2.0,2.0,3.0,2.0,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,2.0,1.0,0.0,1.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.0,0,2.4,1.25,0.444444,1.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000
1,1004,52,1,13,6,1,3,1,4,3,0,1.0,1.0,0.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,3.0,1.0,2.0,0.0,1.0,0.0,0.0,0.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,1,1.0,2.00,0.444444,1.50,0.0,0.0,0.000000,0.0,-0.333333,0.0,-0.5,0.000000,-0.333333,0.000000,-0.333333,-0.5,-0.666667,-0.333333,-0.25,-0.5,-0.333333,0.0,-0.5,0.0,0.0,0.0,-0.333333,-0.5,-0.5,-0.5,-0.5,0.0,0.0,0.000000,0.0,0.0,0.000000,1.0,0.0,-0.2,-0.250000,-0.307692,-0.400
2,1006,71,1,10,19,0,0,1,0,2,0,1.0,3.0,0.0,1.0,1.0,0.0,0.0,0.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,1.0,0,2.0,0,0.4,1.25,0.222222,1.25,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000
3,1007,75,1,18,9,0,0,2,3,1,2,0.0,0.0,2.0,1.0,1.0,1.0,0.0,2.0,2.0,2.0,1.0,2.0,1.0,3.0,3.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,2.0,0,2.0,1,1.0,2.50,0.555556,1.50,1.0,0.0,-0.666667,-0.5,-0.500000,-0.5,0.0,-0.333333,0.000000,-0.333333,0.000000,0.0,-0.500000,-0.750000,-0.25,0.0,0.500000,0.0,0.0,0.0,0.0,-0.5,-0.333333,0.0,0.0,-0.5,-0.5,-0.5,0.0,-0.333333,0.0,0.0,-0.333333,0.0,0.0,-0.5,-0.357143,-0.285714,-0.200
4,1016,66,0,15,2,0,0,0,0,0,0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,2.0,1,0.0,1.00,0.111111,1.00,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.000000,-0.500000,-0.500000,0.5,0.000000,0.

In [349]:
clinical_data.to_csv('cispd_clinical_preprocessed.csv',index=False)